# Imports and preparation

In [1]:
import os

In [2]:
import numpy as np

In [3]:
import pandas as pd

In [4]:
from tqdm import tqdm

In [5]:
import time, gc

In [6]:
from itertools import product


In [7]:
from util import util_elastic

OpenAI tiktoken module is not available for Python < 3.8,Linux ARM64 and AARCH64. Falling back to GPT2TokenizerFast.


In [8]:
from util import util_search

In [9]:
# Para ter repetibilidade nos resultados
random_state = 1

# Tratar valores infinitos (+ e -) como np.NaN
pd.options.mode.use_inf_as_na = True

# IMPORTANTE para tornar figuras interativas
# %matplotlib notebook

# Tamanho padrão das figuras
figsize=(10,6)

pd.set_option('display.max_row', 1000)

pd.set_option('display.max_columns', 50)

pd.set_option('display.column_space', 40)
pd.set_option('display.max_colwidth', 100)
pd.set_option('display.width', 200)


In [10]:
PATH_QUERY = '../data/juris_tcu_index/query.csv'
PATH_QREL =  '../data/juris_tcu_index/qrel.csv'

# Data load

## Query data load

In [12]:
df_query = pd.read_csv(PATH_QUERY)

In [13]:
df_query.shape

(16045, 11)

In [14]:
df_query[df_query['ID']==2845]

,ID,TEXT,REFERENCE_LIST,PARADIGMATIC,AREA_NAME,AREA_ID_DESCRIPTOR,NORMATIVE_PROCESS_TYPE,NORMATIVE_IDENTIFICATION,NORMATIVE_DATE,NORMATIVE_AUTHOR_TYPE,NORMATIVE_AUTHOR_NAME
5618,2845,"Em regra, o pregão é a modalidade de licitação adequada para a concessão remunerada de uso de be...",NaN,NaN,Licitação,932,REPRESENTAÇÃO,Acórdão 478/2016 - Plenário,2016-03-02,RELATOR,MARCOS BEMQUERER


In [15]:
df_query.head()

,ID,TEXT,REFERENCE_LIST,PARADIGMATIC,AREA_NAME,AREA_ID_DESCRIPTOR,NORMATIVE_PROCESS_TYPE,NORMATIVE_IDENTIFICATION,NORMATIVE_DATE,NORMATIVE_AUTHOR_TYPE,NORMATIVE_AUTHOR_NAME
0,13568,"É ilegal a contagem de tempo ficto de serviço prestado sob a égide da Lei 3.313/1957, proporcion...",Lei Complementar 51/1985 || Lei Ordinária 3313/1957,NaN,Pessoal,1131,APOSENTADORIA,Acórdão 168/2014 - Segunda Câmara,2014-01-28,RELATOR,JOSÉ JORGE
1,11614,"SÚMULA TCU 283: Para fim de habilitação, a Administração Pública não deve exigir dos licitantes ...",NaN,SUMULA,Licitação,932,ADMINISTRATIVO,Acórdão 1613/2013 - Plenário,2013-06-26,RELATOR,JOSÉ JORGE
2,21087,"A contratação de serviços por preços superiores às referências legais (Sinapi e Sicro) deve ser,...",NaN,NaN,Licitação,932,PRESTAÇÃO DE CONTAS,Acórdão 3936/2013 - Segunda Câmara,2013-07-09,RELATOR,ANA ARRAES
3,35016,"Não se aplica, excepcionalmente, multa aos gestores por grave infração à norma legal se a irregu...",NaN,NaN,Responsabilidade,775,REPRESENTAÇÃO,Acórdão 3231/2011 - Plenário,2011-12-07,RELATOR,AROLDO CEDRAZ
4,29370,"Em contratatações de TI, não cabe aceitar propostas de preço com incidência de encargos majorado...",NaN,NaN,Licitação,932,REPRESENTAÇÃO,Acórdão 3231/2011 - Plenário,2011-12-07,RELATOR,AROLDO CEDRAZ


## Qrel data load

In [16]:
df_qrel = pd.read_csv(PATH_QREL)

In [17]:
df_qrel.shape

(94809, 3)

In [18]:
df_qrel.head()

,ID_QUERY,ID_DOCTO,TYPE
0,158,15147,INDEXACAO_EXTRA
1,37,15147,INDEXACAO_EXTRA
2,178,15147,INDEXACAO_EXTRA
3,14564,15147,INDEXACAO_EXTRA
4,9219,15147,INDEXACAO_EXTRA


In [19]:
df_qrel.shape[0]

94809

In [20]:
df_search_data = df_query.merge(df_qrel, how='left', left_on='ID', right_on='ID_QUERY').drop('ID_QUERY', axis=1)

In [21]:
df_search_data.shape[0]


94809

In [22]:
# Agrupar os dados pelo ID e criar a coluna 'RELEVANCE_LIST' com a lista de tuplas
# df_new = df_search_data.groupby('ID').apply(lambda x: list(zip(x['ID_DOCTO'], x['TYPE']))).reset_index(name='RELEVANCE_LIST')
# df_new['RELEVANCE_LIST'] = df_new['RELEVANCE_LIST'].apply(lambda x: sorted(x, key=lambda tup: ('AREA', 'TEMA', 'SUBTEMA', 'INDEXACAO_EXTRA').index(tup[1])))
df_new = df_search_data.groupby('ID').apply(lambda x: dict(zip(x['ID_DOCTO'], x['TYPE']))).reset_index(name='RELEVANCE_DICT')


In [23]:
df_new.head()

,ID,RELEVANCE_DICT
0,5,"{1298: 'INDEXACAO_EXTRA', 15340: 'INDEXACAO_EXTRA', 15961: 'INDEXACAO_EXTRA', 1131: 'AREA', 5106..."
1,6,"{207: 'INDEXACAO_EXTRA', 15961: 'INDEXACAO_EXTRA', 1131: 'AREA', 5106: 'SUBTEMA', 1113: 'TEMA'}"
2,7,"{108: 'INDEXACAO_EXTRA', 1526: 'TEMA', 1727: 'SUBTEMA', 223: 'INDEXACAO_EXTRA', 650: 'INDEXACAO_..."
3,8,"{887: 'SUBTEMA', 92: 'TEMA', 1131: 'AREA', 1100820: 'INDEXACAO_EXTRA'}"
4,9,"{1215: 'INDEXACAO_EXTRA', 14715: 'SUBTEMA', 46: 'TEMA', 1480: 'INDEXACAO_EXTRA', 5095: 'AREA'}"


In [24]:
# Unir o resultado com as demais colunas únicas
df_new = pd.merge(df_new, df_search_data.drop_duplicates('ID'), on='ID', how='left')

In [25]:

# Selecionar as colunas desejadas
df_search_data = df_new[['ID', 'TEXT', 'REFERENCE_LIST', 'PARADIGMATIC', 'AREA_NAME', 'AREA_ID_DESCRIPTOR', 'NORMATIVE_PROCESS_TYPE', 'NORMATIVE_IDENTIFICATION', 'NORMATIVE_DATE', 'NORMATIVE_AUTHOR_TYPE', 'NORMATIVE_AUTHOR_NAME', 'RELEVANCE_DICT']]


In [26]:
df_search_data.shape

(16045, 12)

In [27]:
df_search_data = df_search_data.reset_index()

In [28]:
df_search_data.head()

,index,ID,TEXT,REFERENCE_LIST,PARADIGMATIC,AREA_NAME,AREA_ID_DESCRIPTOR,NORMATIVE_PROCESS_TYPE,NORMATIVE_IDENTIFICATION,NORMATIVE_DATE,NORMATIVE_AUTHOR_TYPE,NORMATIVE_AUTHOR_NAME,RELEVANCE_DICT
0,0,5,"SÚMULA TCU 1: Não se compreendem como vencimento, para efeito de concessão da pensão especial co...","Lei Ordinária 3738/1960 || Lei Ordinária 1711/1952, art. 184",SUMULA,Pessoal,1131,NaN,Ata 88/1973 - Plenário,1973-12-04,RELATOR,OCTÁVIO GALLOTTI,"{1298: 'INDEXACAO_EXTRA', 15340: 'INDEXACAO_EXTRA', 15961: 'INDEXACAO_EXTRA', 1131: 'AREA', 5106..."
1,1,6,"SÚMULA TCU 2: Configura-se como vencimento, para efeito da concessão da pensão especial com fund...",Lei Ordinária 3738/1960,SUMULA,Pessoal,1131,NaN,Ata 88/1973 - Plenário,1973-12-04,RELATOR,OCTÁVIO GALLOTTI,"{207: 'INDEXACAO_EXTRA', 15961: 'INDEXACAO_EXTRA', 1131: 'AREA', 5106: 'SUBTEMA', 1113: 'TEMA'}"
2,2,7,SÚMULA TCU 3: O arquivamento é a solução indicada para as hipóteses em que as contas de responsá...,NaN,SUMULA,Direito processual,5288,NaN,Ata 88/1973 - Plenário,1973-12-04,RELATOR,OCTÁVIO GALLOTTI,"{108: 'INDEXACAO_EXTRA', 1526: 'TEMA', 1727: 'SUBTEMA', 223: 'INDEXACAO_EXTRA', 650: 'INDEXACAO_..."
3,3,8,"SÚMULA TCU 4: A reclassificação de cargos não aproveita ao servidor aposentado, a menos que lei ...",NaN,SUMULA,Pessoal,1131,NaN,Ata 88/1973 - Plenário,1973-12-04,RELATOR,OCTÁVIO GALLOTTI,"{887: 'SUBTEMA', 92: 'TEMA', 1131: 'AREA', 1100820: 'INDEXACAO_EXTRA'}"
4,4,9,"SÚMULA TCU 5 (REVOGADA): As sociedades de economia mista, salvo disposição expressa em lei, não ...",NaN,SUMULA,Competência do TCU,5095,NaN,AC 2082/2007-PL,2007-10-03,RELATOR,UBIRATAN AGUIAR,"{1215: 'INDEXACAO_EXTRA', 14715: 'SUBTEMA', 46: 'TEMA', 1480: 'INDEXACAO_EXTRA', 5095: 'AREA'}"


In [29]:
del df_new

# Create reference to index (elastic search)

Documentation https://docs.haystack.deepset.ai/docs
https://haystack.deepset.ai/tutorials/09_dpr_training 


In [30]:
index_dict = util_elastic.return_indexes('indir', parm_print=True)

Index: indir_juris_tcu_index
{'health': 'yellow', 'status': 'open', 'index': 'indir_juris_tcu_index', 'uuid': 'Pl89T40dSwORJnIuYF1t9A', 'pri': '1', 'rep': '1', 'docs.count': '13255', 'docs.deleted': '13255', 'store.size': '289.1mb', 'pri.store.size': '289.1mb'}



In [31]:
index_dict

{'indir_juris_tcu_index': {'health': 'yellow',
  'status': 'open',
  'index': 'indir_juris_tcu_index',
  'uuid': 'Pl89T40dSwORJnIuYF1t9A',
  'pri': '1',
  'rep': '1',
  'docs.count': '13255',
  'docs.deleted': '13255',
  'store.size': '289.1mb',
  'pri.store.size': '289.1mb'}}

In [32]:
INDEX_NAME = 'indir_juris_tcu_index'

In [33]:
index = util_elastic.return_index(parm_index_name=INDEX_NAME)


Qtd de documentos 13255

Qtd de embeddings 13255

Documento.id=1: <Document: id=1, content='O termo é "Abandono de cargo".
Abandono de cargo tem definição: "Configura abandono de cargo a ausên...'>


# Pipelines creation

In [83]:
pipes = []

In [35]:
# Test query
# parm_query = "trata-se de uma denúncia contra o prefeito de Manhuaçu por não haver pago os funcionários da área de limpeza urbana"
parm_query = "A transferência de documentos da entidade para local impróprio ao armazenamento, causando a perda de informações ou inviabilizando seu manuseio, de forma a impedir a atuação do TCU, é causa de responsabilização do gestor que a ordenou."

In [36]:
nome_modelo_monot5_3b = 'unicamp-dl/mt5-3B-mmarco-en-pt'
# "A mono-ptT5 reranker model (850 mb) pretrained in the BrWac corpus, finetuned for 100k steps on Portuguese translated version of MS MARCO passage dataset. The portuguese dataset was translated using Google Translate.")
nome_caminho_modelo_monot5_3b = "/home/borela/fontes/relevar-busca/modelo/" + nome_modelo_monot5_3b
assert os.path.exists(nome_caminho_modelo_monot5_3b), f"Path para {nome_caminho_modelo_monot5_3b} não existe!"

In [37]:
nome_modelo_ranking_minilm = 'unicamp-dl/mMiniLM-L6-v2-pt-v2'
nome_caminho_modelo_minilm = "/home/borela/fontes/relevar-busca/modelo/" + nome_modelo_ranking_minilm
assert os.path.exists(nome_caminho_modelo_minilm), f"Path para {nome_caminho_modelo_minilm} não existe!"

In [38]:
nome_modelo_embedding_model_sts = "rufimelo/Legal-BERTimbau-sts-large-ma-v3"
nome_caminho_modelo_sts = "/home/borela/fontes/relevar-busca/modelo/" + nome_modelo_embedding_model_sts
assert os.path.exists(nome_caminho_modelo_sts), f"Path para {nome_caminho_modelo_sts} não existe!"


In [39]:
raise Exception ('Stop execution - create only desired pipelines in code below ')

Exception: Stop execution - create only desired pipelines in code below 

## sts_multihop

In [40]:
import importlib
importlib.reload(util_search)

<module 'util.util_search' from '/home/borela/fontes/ind-ir/code/util/util_search.py'>

In [41]:
%%time
pipe_sts_multihop_ranker_monot5_3b = util_search.return_pipeline_sts_multihop_reranker(index, 'MONOT5', 
parm_path_model_ranker=nome_caminho_modelo_monot5_3b, parm_path_model_sts=nome_caminho_modelo_sts, parm_limit_query_size=350)

CPU times: user 39.1 s, sys: 7.85 s, total: 47 s
Wall time: 36.8 s


In [84]:
pipes.append({'PIPE_NAME': 'pipe_sts_multihop_ranker_monot5_3b',
              'PIPE_OBJECT': pipe_sts_multihop_ranker_monot5_3b,
              'RETRIEVER_TYPE': 'sts_multihop',  # in ['sts', 'bm25']
              'RETRIEVER_MODEL_NAME': nome_modelo_embedding_model_sts, # or nome_modelo_embedding_model_sts_rufimelo,
              'RANKER_MODEL_NAME': nome_modelo_monot5_3b})

In [77]:
%%time
doctos_retornados_ranker = pipe_sts_multihop_ranker_monot5_3b.run(query=parm_query, params= {'Retriever': {'filters':{'class':['Termo']}}}) 
util_search.detail_document_found(doctos_retornados_ranker)

Parâmetros usados: {'Retriever': {'filters': {'class': ['Termo']}}}
Consulta: A transferência de documentos da entidade para local impróprio ao armazenamento, causando a perda de informações ou inviabilizando seu manuseio, de forma a impedir a atuação do TCU, é causa de responsabilização do gestor que a ordenou.
Qtd documentos retornados: 10
Primeiro docto:
<Document: id=5098, content='O termo é "Responsabilidade perante o controle externo".
Responsabilidade perante o controle externo...'>

Último (10):
<Document: id=5450, content='O termo é "Dispensa sem justa causa".
Dispensa sem justa causa tem definição: "Art. 477.  Na extinçã...'>
Seguem os nomes dos termos recuperados em ordem de score
0 : ['Responsabilidade perante o controle externo', -0.29318735003471375]
1 : ['Afastamento de responsável', -0.8160551190376282]
2 : ['Descarte de documentos', -1.4474653005599976]
3 : ['Excludente de responsabilidade', -2.6080782413482666]
4 : ['Perda de objeto', -2.8628034591674805]
5 : ['Afasta

## Join

In [ ]:
import importlib
importlib.reload(util_search)

<module 'util.util_search' from '/home/borela/fontes/ind-ir/code/util/util_search.py'>

In [ ]:
%%time
pipe_join_ranker_monot5_3b = util_search.return_pipeline_join_reranker(index, 'MONOT5', 
parm_path_model_ranker=nome_caminho_modelo_monot5_3b, parm_path_model_sts=nome_caminho_modelo_sts, parm_limit_query_size=350)

CPU times: user 38.6 s, sys: 7.72 s, total: 46.4 s
Wall time: 36 s


imprimir o pipe

In [ ]:
from pathlib import Path

In [ ]:
pipe_join_ranker_monot5_3b.draw(Path("pipe_join_ranker_monot5_3b.png"))	 

In [ ]:
pipe_join_ranker_monot5_3b.save_to_yaml(Path("pipe_join_ranker_monot5_3b.yahml"), return_defaults = True)


In [ ]:
pipe_join_ranker_monot5_3b.get_config(return_defaults=True)


In [ ]:
pipes.append({'PIPE_NAME': 'pipe_join_ranker_monot5_3b',
              'PIPE_OBJECT': pipe_join_ranker_monot5_3b,
              'RETRIEVER_TYPE': 'join',  # in ['sts', 'bm25']
              'RETRIEVER_MODEL_NAME': nome_modelo_embedding_model_sts, # or nome_modelo_embedding_model_sts_rufimelo,
              'RANKER_MODEL_NAME': nome_modelo_monot5_3b})

In [ ]:
%%time
doctos_retornados_ranker = pipe_join_ranker_monot5_3b.run(query=parm_query, 
 params={"Bm25Retriever": {"top_k": 300}, "StsRetriever": {"top_k": 100}, "Ranker": {"top_k": 10}})
util_search.detail_document_found(doctos_retornados_ranker)

ValueError: No node(s) or global parameter(s) named Retriever found in pipeline.

join

In [ ]:
%%time
pipe_join = util_search.return_pipeline_join(index, parm_path_model_sts=nome_caminho_modelo_sts)

CPU times: user 3.44 s, sys: 653 ms, total: 4.1 s
Wall time: 2.15 s


In [ ]:
parm_query

'A transferência de documentos da entidade para local impróprio ao armazenamento, causando a perda de informações ou inviabilizando seu manuseio, de forma a impedir a atuação do TCU, é causa de responsabilização do gestor que a ordenou.'

In [ ]:
%%time
doctos_retornados_ranker = pipe_join.run(query=parm_query, 
 params={"Bm25Retriever": {"top_k": 300}, "StsRetriever": {"top_k": 100}})
util_search.detail_document_found(doctos_retornados_ranker)

Parâmetros usados: {'Bm25Retriever': {'top_k': 300}, 'StsRetriever': {'top_k': 100}}
Consulta: A transferência de documentos da entidade para local impróprio ao armazenamento, causando a perda de informações ou inviabilizando seu manuseio, de forma a impedir a atuação do TCU, é causa de responsabilização do gestor que a ordenou.
Qtd documentos retornados: 379
Primeiro docto:
<Document: id=1102005, content='O termo é "Plano de contratações".
Plano de contratações tem definição: "É o documento no qual a org...'>

Último (379):
<Document: id=2992, content='O termo é "Afastamento preventivo".
Afastamento preventivo tem definição: "Art. 147.  Como medida ca...'>
Seguem os nomes dos termos recuperados em ordem de score
0 : ['Plano de contratações', 0.9856940203493179]
1 : ['Competência do TCU', 0.9849874794705751]
2 : ['Teletrabalho', 0.9823437136689326]
3 : ['Nexo de causalidade', 0.982157658932855]
4 : ['Processo administrativo de responsabilização', 0.9816688772992885]
5 : ['Evidência doc

## First stage = BM25

### pipe_bm25_ranker_monot5_3b

In [ ]:
%%time
pipe_bm25_ranker_monot5_3b = util_search.return_pipeline_bm25_reranker(index, 'MONOT5', nome_caminho_modelo_monot5_3b, parm_limit_query_size=350)

CPU times: user 34.8 s, sys: 7.17 s, total: 42 s
Wall time: 34.9 s


In [ ]:
pipes.append({'PIPE_NAME': 'pipe_bm25_ranker_monot5_3b',
              'PIPE_OBJECT': pipe_bm25_ranker_monot5_3b,
              'RETRIEVER_TYPE': 'bm25',  # or 'sts'
              'RETRIEVER_MODEL_NAME': '', # or nome_modelo_embedding_model_sts_rufimelo,
              'RANKER_MODEL_NAME': nome_modelo_monot5_3b+'-tokens-extra'})
              #'RANKER_MODEL_NAME': nome_modelo_monot5_3b})

In [ ]:
doctos_retornados_ranker = pipe_bm25_ranker_monot5_3b.run(query=parm_query)
util_search.detail_document_found(doctos_retornados_ranker)

Parâmetros usados: {}
Consulta: A transferência de documentos da entidade para local impróprio ao armazenamento, causando a perda de informações ou inviabilizando seu manuseio, de forma a impedir a atuação do TCU, é causa de responsabilização do gestor que a ordenou.
Qtd documentos retornados: 10
Primeiro docto:
<Document: id=1101364, content='O termo é "Evidência documental".
Evidência documental tem definição: "Evidência de auditoria obtida...'>

Último (10):
<Document: id=1102005, content='O termo é "Plano de contratações".
Plano de contratações tem definição: "É o documento no qual a org...'>
Seguem os nomes dos termos recuperados em ordem de score
0 : ['Evidência documental', -50.622467041015625]
1 : ['Competência do TCU', -51.267578125]
2 : ['Processo administrativo de responsabilização', -51.291893005371094]
3 : ['Ofício de requisição', -52.43625259399414]
4 : ['Julgamento de contas', -55.70490264892578]
5 : ['Teletrabalho', -58.83453369140625]
6 : ['Nexo de causalidade', -58.90

In [ ]:
doctos_retornados_ranker = pipe_bm25_ranker_monot5_3b.run(query=parm_query)
util_search.detail_document_found(doctos_retornados_ranker)

Parâmetros usados: {}
Consulta: A transferência de documentos da entidade para local impróprio ao armazenamento, causando a perda de informações ou inviabilizando seu manuseio, de forma a impedir a atuação do TCU, é causa de responsabilização do gestor que a ordenou.
Qtd documentos retornados: 10
Primeiro docto:
<Document: id=1104189, content='O termo é "Processo administrativo de responsabilização". Processo administrativo de responsabilizaç...'>

Último (10):
<Document: id=1102005, content='O termo é "Plano de contratações".
Plano de contratações tem definição: "É o documento no qual a org...'>
Seguem os nomes dos termos recuperados em ordem de score
0 : ['Processo administrativo de responsabilização', -1.0603076219558716]
1 : ['Competência do TCU', -1.4214444160461426]
2 : ['Evidência documental', -1.9571717977523804]
3 : ['Ofício de requisição', -2.266594648361206]
4 : ['Julgamento de contas', -2.5948143005371094]
5 : ['Nexo de causalidade', -3.0983963012695312]
6 : ['Teletrabalho'

### pipe_bm25_ranker_minilm

In [ ]:
pipe_bm25_ranker_minilm = util_search.return_pipeline_bm25_reranker(index, 'MINILM', nome_caminho_modelo_minilm, parm_limit_query_size=350)

In [ ]:
pipes.append({'PIPE_NAME': 'pipe_bm25_ranker_minilm',
              'PIPE_OBJECT': pipe_bm25_ranker_minilm,
              'RETRIEVER_TYPE': 'bm25',  # or 'sts'
              'RETRIEVER_MODEL_NAME': '', # or nome_modelo_embedding_model_sts_rufimelo,
              'RANKER_MODEL_NAME': nome_modelo_ranking_minilm})

In [ ]:
doctos_retornados_ranker = pipe_bm25_ranker_minilm.run(query=parm_query)
util_search.detail_document_found(doctos_retornados_ranker)

Parâmetros usados: {}
Consulta: A transferência de documentos da entidade para local impróprio ao armazenamento, causando a perda de informações ou inviabilizando seu manuseio, de forma a impedir a atuação do TCU, é causa de responsabilização do gestor que a ordenou.
Qtd documentos retornados: 10
Primeiro docto:
<Document: id=5095, content='O termo é "Competência do TCU". Competência do TCU tem definição: "São as seguintes as competências ...'>

Último (10):
<Document: id=4495, content='O termo é "Nexo de causalidade".
Nexo de causalidade tem definição: "Elemento que evidencia se a con...'>
Seguem os nomes dos termos recuperados em ordem de score
0 : ['Competência do TCU', 0.0014646538766101003]
1 : ['Processo administrativo de responsabilização', 0.0004114470211789012]
2 : ['Evidência documental', 0.00033812460605986416]
3 : ['Plano de contratações', 0.0002457168884575367]
4 : ['Julgamento de contas', 0.00022130725847091526]
5 : ['Teletrabalho', 0.00016929447883740067]
6 : ['Ofício de

## First stage = Sentence Similarity

### pipe_sts_ranker_minilm

In [ ]:
pipe_sts_ranker_minilm = util_search.return_pipeline_sts_reranker(index, 'MINILM', parm_path_model_ranker=nome_caminho_modelo_minilm, parm_path_model_sts=nome_caminho_modelo_sts, parm_limit_query_size=350)

In [ ]:
pipes.append({'PIPE_NAME': 'pipe_sts_ranker_minilm',
              'PIPE_OBJECT': pipe_sts_ranker_minilm,
              'RETRIEVER_TYPE': 'sts',  # in ['sts', 'bm25']
              'RETRIEVER_MODEL_NAME': nome_modelo_embedding_model_sts, # or nome_modelo_embedding_model_sts_rufimelo,
              'RANKER_MODEL_NAME': nome_modelo_ranking_minilm})

In [ ]:
doctos_retornados_ranker = pipe_sts_ranker_minilm.run(query=parm_query)
util_search.detail_document_found(doctos_retornados_ranker)

Parâmetros usados: {}
Consulta: A transferência de documentos da entidade para local impróprio ao armazenamento, causando a perda de informações ou inviabilizando seu manuseio, de forma a impedir a atuação do TCU, é causa de responsabilização do gestor que a ordenou.
Qtd documentos retornados: 10
Primeiro docto:
<Document: id=1110487, content='O termo é "Vazamento de dados".
Vazamento de dados tem definição: "Transmissão não-autorizada de dad...'>

Último (10):
<Document: id=15939, content='O termo é "Erro de procedimento".
Erro de procedimento tem definição: "É um vício de forma, extrínse...'>
Seguem os nomes dos termos recuperados em ordem de score
0 : ['Vazamento de dados', 0.0024509401991963387]
1 : ['Termo de sigilo', 0.0007997120846994221]
2 : ['Trancamento das contas', 0.0002845456183422357]
3 : ['Risco de controle', 0.00022979704954195768]
4 : ['Revisão de ofício', 0.00013715452223550528]
5 : ['Responsabilidade perante o controle externo', 0.00013104191748425364]
6 : ['Anulação

### pipe_sts_ranker_monot5_3b

In [ ]:
pipe_sts_ranker_monot5_3b = util_search.return_pipeline_sts_reranker(index, 'MONOT5', parm_path_model_ranker=nome_caminho_modelo_monot5_3b, parm_path_model_sts=nome_caminho_modelo_sts, parm_limit_query_size=350)

In [ ]:
pipes.append({'PIPE_NAME': 'pipe_sts_ranker_monot5_3b',
              'PIPE_OBJECT': pipe_sts_ranker_monot5_3b,
              'RETRIEVER_TYPE': 'sts',  # in ['sts', 'bm25']
              'RETRIEVER_MODEL_NAME': nome_modelo_embedding_model_sts, # or nome_modelo_embedding_model_sts_rufimelo,
              'RANKER_MODEL_NAME': nome_modelo_monot5_3b})

In [ ]:
doctos_retornados_ranker = pipe_sts_ranker_monot5_3b.run(query=parm_query)
util_search.detail_document_found(doctos_retornados_ranker)

Parâmetros usados: {}
Consulta: A transferência de documentos da entidade para local impróprio ao armazenamento, causando a perda de informações ou inviabilizando seu manuseio, de forma a impedir a atuação do TCU, é causa de responsabilização do gestor que a ordenou.
Qtd documentos retornados: 10
Primeiro docto:
<Document: id=5098, content='O termo é "Responsabilidade perante o controle externo".
Responsabilidade perante o controle externo...'>

Último (10):
<Document: id=83, content='O termo é "Anulação". Anulação tem definição: "É o ato ou a decisão, de caráter judicial ou administ...'>
Seguem os nomes dos termos recuperados em ordem de score
0 : ['Responsabilidade perante o controle externo', -0.29301929473876953]
1 : ['Termo de sigilo', -0.6637465357780457]
2 : ['Afastamento de responsável', -0.8156272172927856]
3 : ['Revisão de ofício', -0.9923231601715088]
4 : ['Trancamento das contas', -1.1499707698822021]
5 : ['Erro de procedimento', -2.8979315757751465]
6 : ['Vazamento de dado

# Experiments

## Rodar monoT5_3b sts_multihop pipe 

In [104]:
pipes

[{'PIPE_NAME': 'pipe_sts_multihop_ranker_monot5_3b',
  'PIPE_OBJECT': <haystack.pipelines.base.Pipeline at 0x7f425949f5d0>,
  'RETRIEVER_TYPE': 'sts_multihop',
  'RETRIEVER_MODEL_NAME': 'rufimelo/Legal-BERTimbau-sts-large-ma-v3',
  'RANKER_MODEL_NAME': 'unicamp-dl/mt5-3B-mmarco-en-pt'}]

In [117]:
grid_experiment = {'CRITERIA' : ["area", "theme", "subtheme","extra"],
             'TOPK_RETRIEVER' : [300],
             'TOPK_RANKER' : [0],
             'PIPE': pipes,
             'DONE': [False]
}

In [118]:
list_experiment = [dict(zip(grid_experiment.keys(), values)) for values in product(*grid_experiment.values())]


In [119]:
print(len(list_experiment))
gc.collect()

4


62

In [120]:
import importlib
importlib.reload(util_search)

<module 'util.util_search' from '/home/borela/fontes/ind-ir/code/util/util_search.py'>

In [121]:
%%time
# colocar pequeno só para testes, depois voltar para 9999999 (número maior do que o número de termos a pesquisar)
limit_query = 100 # 9999999
for cnt, experiment in enumerate(list_experiment):
    if not experiment['DONE']:
        list_result_experiment = [] # por experiment
        # no caso de normas segecex, bastaria 224 para qtd5 e 891 para qtd1
        print(f"==={cnt}/{len(list_experiment)}===>  {time.strftime('%Y-%b-%d %H:%M:%S')} experiment: {experiment}")
        result_run = util_search.experiment_run(parm_df=df_search_data, 
                                                parm_experiment=experiment,
                                                parm_ndcg_position=12,
                                                parm_limit_query=limit_query,
                                                parm_print=True)
        list_result_experiment.append(result_run)
        # print(f"qtd_encontrado sim:{result_run['qtd_encontrado']}, não:{result_run['qtd_nao_encontrado']} ({result_run['percent_nao_encontrado']}%)")
        experiment['DONE'] = True
        util_search.add_experiment_result(parm_list_result=list_result_experiment, parm_dataset='juris_tcu_index')



===0/4===>  2023-Jun-16 21:05:20 experiment: {'CRITERIA': 'area', 'TOPK_RETRIEVER': 300, 'TOPK_RANKER': 0, 'PIPE': {'PIPE_NAME': 'pipe_sts_multihop_ranker_monot5_3b', 'PIPE_OBJECT': <haystack.pipelines.base.Pipeline object at 0x7f425949f5d0>, 'RETRIEVER_TYPE': 'sts_multihop', 'RETRIEVER_MODEL_NAME': 'rufimelo/Legal-BERTimbau-sts-large-ma-v3', 'RANKER_MODEL_NAME': 'unicamp-dl/mt5-3B-mmarco-en-pt'}, 'DONE': False}


 99%|█████████▉| 99/100 [01:13<00:00,  1.34it/s]


RANK1_MEAN: 3.63
NDCG_MEAN: 62.094
TIME_SPENT_MEAN: 0.74
===1/4===>  2023-Jun-16 21:06:35 experiment: {'CRITERIA': 'theme', 'TOPK_RETRIEVER': 300, 'TOPK_RANKER': 0, 'PIPE': {'PIPE_NAME': 'pipe_sts_multihop_ranker_monot5_3b', 'PIPE_OBJECT': <haystack.pipelines.base.Pipeline object at 0x7f425949f5d0>, 'RETRIEVER_TYPE': 'sts_multihop', 'RETRIEVER_MODEL_NAME': 'rufimelo/Legal-BERTimbau-sts-large-ma-v3', 'RANKER_MODEL_NAME': 'unicamp-dl/mt5-3B-mmarco-en-pt'}, 'DONE': False}


 52%|█████▏    | 52/100 [18:14<16:50, 21.05s/it]


KeyboardInterrupt: 

## Rodar monoT5_3b join pipe 

In [ ]:
pipes

[{'PIPE_NAME': 'pipe_join_ranker_monot5_3b',
  'PIPE_OBJECT': <haystack.pipelines.base.Pipeline at 0x7fb44f0d1610>,
  'RETRIEVER_TYPE': 'join',
  'RETRIEVER_MODEL_NAME': 'rufimelo/Legal-BERTimbau-sts-large-ma-v3',
  'RANKER_MODEL_NAME': '/home/borela/fontes/relevar-busca/modelo/unicamp-dl/mt5-3B-mmarco-en-pt'}]

In [ ]:
grid_experiment = {'CRITERIA' : ['total'],
             'TOPK_RETRIEVER' : [100],
             'TOPK_RANKER' : [100],
             'PIPE': pipes,
             'DONE': [False]
}

In [ ]:
list_experiment = [dict(zip(grid_experiment.keys(), values)) for values in product(*grid_experiment.values())]


In [ ]:
print(len(list_experiment))
gc.collect()

1


71

In [ ]:
list_experiment

[{'CRITERIA': 'total',
  'TOPK_RETRIEVER': 100,
  'TOPK_RANKER': 100,
  'PIPE': {'PIPE_NAME': 'pipe_join_ranker_monot5_3b',
   'PIPE_OBJECT': <haystack.pipelines.base.Pipeline at 0x7fb44f0d1610>,
   'RETRIEVER_TYPE': 'join',
   'RETRIEVER_MODEL_NAME': 'rufimelo/Legal-BERTimbau-sts-large-ma-v3',
   'RANKER_MODEL_NAME': '/home/borela/fontes/relevar-busca/modelo/unicamp-dl/mt5-3B-mmarco-en-pt'},
  'DONE': False}]

In [ ]:
import importlib
importlib.reload(util_search)

<module 'util.util_search' from '/home/borela/fontes/ind-ir/code/util/util_search.py'>

In [ ]:
%%time
# colocar pequeno só para testes, depois voltar para 9999999 (número maior do que o número de termos a pesquisar)
limit_query = 100 # 9999999
for cnt, experiment in enumerate(list_experiment):
    if not experiment['DONE']:
        list_result_experiment = [] # por experiment
        # no caso de normas segecex, bastaria 224 para qtd5 e 891 para qtd1
        print(f"==={cnt}/{len(list_experiment)}===>  {time.strftime('%Y-%b-%d %H:%M:%S')} experiment: {experiment}")
        result_run = util_search.experiment_run(parm_df=df_search_data, 
                                                parm_experiment=experiment,
                                                parm_ndcg_position=12,
                                                parm_limit_query=limit_query,
                                                parm_print=True)
        list_result_experiment.append(result_run)
        # print(f"qtd_encontrado sim:{result_run['qtd_encontrado']}, não:{result_run['qtd_nao_encontrado']} ({result_run['percent_nao_encontrado']}%)")
        experiment['DONE'] = True
        util_search.add_experiment_result(parm_list_result=list_result_experiment, parm_dataset='juris_tcu_index')



===0/1===>  2023-Jun-16 13:17:25 experiment: {'CRITERIA': 'total', 'TOPK_RETRIEVER': 100, 'TOPK_RANKER': 100, 'PIPE': {'PIPE_NAME': 'pipe_join_ranker_monot5_3b', 'PIPE_OBJECT': <haystack.pipelines.base.Pipeline object at 0x7fb44f0d1610>, 'RETRIEVER_TYPE': 'join', 'RETRIEVER_MODEL_NAME': 'rufimelo/Legal-BERTimbau-sts-large-ma-v3', 'RANKER_MODEL_NAME': '/home/borela/fontes/relevar-busca/modelo/unicamp-dl/mt5-3B-mmarco-en-pt'}, 'DONE': False}


 99%|█████████▉| 99/100 [40:30<00:24, 24.55s/it]


RANK1_MEAN: 7.5
NDCG_MEAN: 37.202
TIME_SPENT_MEAN: 24.303
CPU times: user 23min 2s, sys: 17min 9s, total: 40min 12s
Wall time: 40min 30s


## Rodar monoT5_3b: bm25 com mais tokens positivas e negativas

### topk_ranker 100

In [ ]:
pipes

[{'PIPE_NAME': 'pipe_bm25_ranker_monot5_3b',
  'PIPE_OBJECT': <haystack.pipelines.base.Pipeline at 0x7fe8f7cef650>,
  'RETRIEVER_TYPE': 'bm25',
  'RETRIEVER_MODEL_NAME': '',
  'RANKER_MODEL_NAME': 'unicamp-dl/mt5-3B-mmarco-en-pt-tokens-extra'}]

In [ ]:
grid_experiment = {'CRITERIA' : ['total'],
             'TOPK_RETRIEVER' : [100],
             'TOPK_RANKER' : [0],
             'PIPE': pipes,
             'DONE': [False]
}

In [ ]:
list_experiment = [dict(zip(grid_experiment.keys(), values)) for values in product(*grid_experiment.values())]


In [ ]:
print(len(list_experiment))
gc.collect()

1


100

In [ ]:
import importlib
importlib.reload(util_search)

dict_idcg_relevance_fixed {1: 1.0, 2: 1.6309297535714575, 3: 2.1309297535714578, 4: 2.5616063116448506, 5: 2.9484591188793923, 6: 3.3046663059874146, 7: 3.637999639320748, 8: 3.953464516106477, 9: 4.254494511770458, 10: 4.543559338088346, 11: 4.8225022837394755, 12: 5.092740438166795, 13: 5.355389973203989, 14: 5.611347998013804}


<module 'util.util_search' from '/home/borela/fontes/ind-ir/code/util/util_search.py'>

In [ ]:
%%time
# colocar pequeno só para testes, depois voltar para 9999999 (número maior do que o número de termos a pesquisar)
limit_query = 100 # 9999999
for cnt, experiment in enumerate(list_experiment):
    if not experiment['DONE']:
        list_result_experiment = [] # por experiment
        # no caso de normas segecex, bastaria 224 para qtd5 e 891 para qtd1
        print(f"==={cnt}/{len(list_experiment)}===>  {time.strftime('%Y-%b-%d %H:%M:%S')} experiment: {experiment}")
        result_run = util_search.experiment_run(parm_df=df_search_data, 
                                                parm_experiment=experiment,
                                                parm_ndcg_position=12,
                                                parm_limit_query=limit_query,
                                                parm_print=True)
        list_result_experiment.append(result_run)
        # print(f"qtd_encontrado sim:{result_run['qtd_encontrado']}, não:{result_run['qtd_nao_encontrado']} ({result_run['percent_nao_encontrado']}%)")
        experiment['DONE'] = True
        util_search.add_experiment_result(parm_list_result=list_result_experiment, parm_dataset='juris_tcu_index')



===0/1===>  2023-Jun-15 13:18:57 experiment: {'CRITERIA': 'total', 'TOPK_RETRIEVER': 100, 'TOPK_RANKER': 100, 'PIPE': {'PIPE_NAME': 'pipe_bm25_ranker_monot5_3b', 'PIPE_OBJECT': <haystack.pipelines.base.Pipeline object at 0x7fe8f7cef650>, 'RETRIEVER_TYPE': 'bm25', 'RETRIEVER_MODEL_NAME': '', 'RANKER_MODEL_NAME': 'unicamp-dl/mt5-3B-mmarco-en-pt-tokens-extra'}, 'DONE': False}


 99%|█████████▉| 99/100 [11:33<00:07,  7.01s/it]


RANK1_MEAN: 6.99
NDCG_MEAN: 30.581
TIME_SPENT_MEAN: 6.936
CPU times: user 6min 49s, sys: 4min 33s, total: 11min 22s
Wall time: 11min 33s


## Rodar monoT5_3b: bm25 com mais tokens positivas

### topk_ranker 100

In [ ]:
pipes

[{'PIPE_NAME': 'pipe_bm25_ranker_monot5_3b',
  'PIPE_OBJECT': <haystack.pipelines.base.Pipeline at 0x7f94008b4390>,
  'RETRIEVER_TYPE': 'bm25',
  'RETRIEVER_MODEL_NAME': '',
  'RANKER_MODEL_NAME': 'unicamp-dl/mt5-3B-mmarco-en-pt-3tokens-yes'}]

In [ ]:
grid_experiment = {'CRITERIA' : ['total'],
             'TOPK_RETRIEVER' : [100],
             'TOPK_RANKER' : [100],
             'PIPE': pipes,
             'DONE': [False]
}

In [ ]:
list_experiment = [dict(zip(grid_experiment.keys(), values)) for values in product(*grid_experiment.values())]


In [ ]:
print(len(list_experiment))
gc.collect()

1


79

In [ ]:
import importlib
importlib.reload(util_search)

dict_idcg_relevance_fixed {1: 1.0, 2: 1.6309297535714575, 3: 2.1309297535714578, 4: 2.5616063116448506, 5: 2.9484591188793923, 6: 3.3046663059874146, 7: 3.637999639320748, 8: 3.953464516106477, 9: 4.254494511770458, 10: 4.543559338088346, 11: 4.8225022837394755, 12: 5.092740438166795, 13: 5.355389973203989, 14: 5.611347998013804}


<module 'util.util_search' from '/home/borela/fontes/ind-ir/code/util/util_search.py'>

In [ ]:
%%time
# colocar pequeno só para testes, depois voltar para 9999999 (número maior do que o número de termos a pesquisar)
limit_query = 100 # 9999999
for cnt, experiment in enumerate(list_experiment):
    if not experiment['DONE']:
        list_result_experiment = [] # por experiment
        # no caso de normas segecex, bastaria 224 para qtd5 e 891 para qtd1
        print(f"==={cnt}/{len(list_experiment)}===>  {time.strftime('%Y-%b-%d %H:%M:%S')} experiment: {experiment}")
        result_run = util_search.experiment_run(parm_df=df_search_data, 
                                                parm_experiment=experiment,
                                                parm_ndcg_position=12,
                                                parm_limit_query=limit_query,
                                                parm_print=True)
        list_result_experiment.append(result_run)
        # print(f"qtd_encontrado sim:{result_run['qtd_encontrado']}, não:{result_run['qtd_nao_encontrado']} ({result_run['percent_nao_encontrado']}%)")
        experiment['DONE'] = True
        util_search.add_experiment_result(parm_list_result=list_result_experiment, parm_dataset='juris_tcu_index')



===0/1===>  2023-Jun-14 15:21:36 experiment: {'CRITERIA': 'total', 'TOPK_RETRIEVER': 100, 'TOPK_RANKER': 100, 'PIPE': {'PIPE_NAME': 'pipe_bm25_ranker_monot5_3b', 'PIPE_OBJECT': <haystack.pipelines.base.Pipeline object at 0x7f94008b4390>, 'RETRIEVER_TYPE': 'bm25', 'RETRIEVER_MODEL_NAME': '', 'RANKER_MODEL_NAME': 'unicamp-dl/mt5-3B-mmarco-en-pt-3tokens-yes'}, 'DONE': False}


 99%|█████████▉| 99/100 [11:45<00:07,  7.13s/it]


RANK1_MEAN: 8.6
NDCG_MEAN: 27.555
TIME_SPENT_MEAN: 7.057
CPU times: user 6min 43s, sys: 4min 48s, total: 11min 31s
Wall time: 11min 46s


## Rodar monoT5_3b: bm25

### topk_ranker 100

In [ ]:
pipes

[{'PIPE_NAME': 'pipe_bm25_ranker_monot5_3b',
  'PIPE_OBJECT': <haystack.pipelines.base.Pipeline at 0x7fd807582b50>,
  'RETRIEVER_TYPE': 'bm25',
  'RETRIEVER_MODEL_NAME': '',
  'RANKER_MODEL_NAME': 'unicamp-dl/mt5-3B-mmarco-en-pt'}]

In [ ]:
grid_experiment = {'CRITERIA' : ['total', 'total_gte_5'],
             'TOPK_RETRIEVER' : [300, 200, 100],
             'TOPK_RANKER' : [100],
             'PIPE': pipes,
             'DONE': [False]
}

In [ ]:
list_experiment = [dict(zip(grid_experiment.keys(), values)) for values in product(*grid_experiment.values())]


In [ ]:
print(len(list_experiment))
gc.collect()

6


103

In [ ]:
import importlib
importlib.reload(util_search)

dict_idcg_relevance_fixed {1: 1.0, 2: 1.6309297535714575, 3: 2.1309297535714578, 4: 2.5616063116448506, 5: 2.9484591188793923, 6: 3.3046663059874146, 7: 3.637999639320748, 8: 3.953464516106477, 9: 4.254494511770458, 10: 4.543559338088346, 11: 4.8225022837394755, 12: 5.092740438166795, 13: 5.355389973203989, 14: 5.611347998013804}


<module 'util.util_search' from '/home/borela/fontes/ind-ir/code/util/util_search.py'>

In [ ]:
%%time
# colocar pequeno só para testes, depois voltar para 9999999 (número maior do que o número de termos a pesquisar)
limit_query = 100 # 9999999
for cnt, experiment in enumerate(list_experiment):
    if not experiment['DONE']:
        list_result_experiment = [] # por experiment
        # no caso de normas segecex, bastaria 224 para qtd5 e 891 para qtd1
        print(f"==={cnt}/{len(list_experiment)}===>  {time.strftime('%Y-%b-%d %H:%M:%S')} experiment: {experiment}")
        result_run = util_search.experiment_run(parm_df=df_search_data, 
                                                parm_experiment=experiment,
                                                parm_ndcg_position=12,
                                                parm_limit_query=limit_query,
                                                parm_print=True)
        list_result_experiment.append(result_run)
        # print(f"qtd_encontrado sim:{result_run['qtd_encontrado']}, não:{result_run['qtd_nao_encontrado']} ({result_run['percent_nao_encontrado']}%)")
        experiment['DONE'] = True
        util_search.add_experiment_result(parm_list_result=list_result_experiment, parm_dataset='juris_tcu_index')



===0/6===>  2023-Jun-11 19:44:36 experiment: {'CRITERIA': 'total', 'TOPK_RETRIEVER': 300, 'TOPK_RANKER': 100, 'PIPE': {'PIPE_NAME': 'pipe_bm25_ranker_monot5_3b', 'PIPE_OBJECT': <haystack.pipelines.base.Pipeline object at 0x7fd807582b50>, 'RETRIEVER_TYPE': 'bm25', 'RETRIEVER_MODEL_NAME': '', 'RANKER_MODEL_NAME': 'unicamp-dl/mt5-3B-mmarco-en-pt'}, 'DONE': False}


 99%|█████████▉| 99/100 [35:03<00:21, 21.25s/it]


RANK1_MEAN: 4.1
NDCG_MEAN: 39.937
TIME_SPENT_MEAN: 21.039
===1/6===>  2023-Jun-11 20:19:40 experiment: {'CRITERIA': 'total', 'TOPK_RETRIEVER': 200, 'TOPK_RANKER': 100, 'PIPE': {'PIPE_NAME': 'pipe_bm25_ranker_monot5_3b', 'PIPE_OBJECT': <haystack.pipelines.base.Pipeline object at 0x7fd807582b50>, 'RETRIEVER_TYPE': 'bm25', 'RETRIEVER_MODEL_NAME': '', 'RANKER_MODEL_NAME': 'unicamp-dl/mt5-3B-mmarco-en-pt'}, 'DONE': False}


 99%|█████████▉| 99/100 [23:58<00:14, 14.53s/it]


RANK1_MEAN: 3.52
NDCG_MEAN: 38.97
TIME_SPENT_MEAN: 14.389
===2/6===>  2023-Jun-11 20:43:39 experiment: {'CRITERIA': 'total', 'TOPK_RETRIEVER': 100, 'TOPK_RANKER': 100, 'PIPE': {'PIPE_NAME': 'pipe_bm25_ranker_monot5_3b', 'PIPE_OBJECT': <haystack.pipelines.base.Pipeline object at 0x7fd807582b50>, 'RETRIEVER_TYPE': 'bm25', 'RETRIEVER_MODEL_NAME': '', 'RANKER_MODEL_NAME': 'unicamp-dl/mt5-3B-mmarco-en-pt'}, 'DONE': False}


...

RANK1_MEAN: 3.16
NDCG_MEAN: 39.915
TIME_SPENT_MEAN: 21.487
===4/6===>  2023-Jun-11 21:31:28 experiment: {'CRITERIA': 'total_gte_5', 'TOPK_RETRIEVER': 200, 'TOPK_RANKER': 100, 'PIPE': {'PIPE_NAME': 'pipe_bm25_ranker_monot5_3b', 'PIPE_OBJECT': <haystack.pipelines.base.Pipeline object at 0x7fd807582b50>, 'RETRIEVER_TYPE': 'bm25', 'RETRIEVER_MODEL_NAME': '', 'RANKER_MODEL_NAME': 'unicamp-dl/mt5-3B-mmarco-en-pt'}, 'DONE': False}


 99%|█████████▉| 99/100 [24:01<00:14, 14.56s/it]


RANK1_MEAN: 2.81
NDCG_MEAN: 40.249
TIME_SPENT_MEAN: 14.419
===5/6===>  2023-Jun-11 21:55:31 experiment: {'CRITERIA': 'total_gte_5', 'TOPK_RETRIEVER': 100, 'TOPK_RANKER': 100, 'PIPE': {'PIPE_NAME': 'pipe_bm25_ranker_monot5_3b', 'PIPE_OBJECT': <haystack.pipelines.base.Pipeline object at 0x7fd807582b50>, 'RETRIEVER_TYPE': 'bm25', 'RETRIEVER_MODEL_NAME': '', 'RANKER_MODEL_NAME': 'unicamp-dl/mt5-3B-mmarco-en-pt'}, 'DONE': False}


 99%|█████████▉| 99/100 [12:06<00:07,  7.33s/it]

RANK1_MEAN: 2.82
NDCG_MEAN: 39.205
TIME_SPENT_MEAN: 7.26
CPU times: user 1h 26min 8s, sys: 55min 59s, total: 2h 22min 7s
Wall time: 2h 23min 1s


### topk_ranker 50

In [ ]:
pipes

[{'PIPE_NAME': 'pipe_bm25_ranker_monot5_3b',
  'PIPE_OBJECT': <haystack.pipelines.base.Pipeline at 0x7f3ff38305d0>,
  'RETRIEVER_TYPE': 'bm25',
  'RETRIEVER_MODEL_NAME': '',
  'RANKER_MODEL_NAME': 'unicamp-dl/mt5-3B-mmarco-en-pt'}]

In [ ]:
grid_experiment = {
             'CRITERIA' : ['total', 'total_gte_5'],
             'TOPK_RETRIEVER' : [400, 300, 200, 100],
             'TOPK_RANKER' : [50],
             'DONE': [False],
             'PIPE': pipes,
}

In [ ]:
list_experiment = [dict(zip(grid_experiment.keys(), values)) for values in product(*grid_experiment.values())]


In [ ]:
print(len(list_experiment))
gc.collect()

8


114

In [ ]:
import importlib
importlib.reload(util_search)

dict_idcg_relevance_fixed {1: 1.0, 2: 1.6309297535714575, 3: 2.1309297535714578, 4: 2.5616063116448506, 5: 2.9484591188793923, 6: 3.3046663059874146, 7: 3.637999639320748, 8: 3.953464516106477, 9: 4.254494511770458, 10: 4.543559338088346, 11: 4.8225022837394755, 12: 5.092740438166795, 13: 5.355389973203989, 14: 5.611347998013804}


<module 'util.util_search' from '/home/borela/fontes/ind-ir/code/util/util_search.py'>

In [ ]:
%%time
# colocar pequeno só para testes, depois voltar para 9999999 (número maior do que o número de termos a pesquisar)
limit_query = 100 # 9999999
for cnt, experiment in enumerate(list_experiment):
    if not experiment['DONE']:
        list_result_experiment = [] # por experiment
        # no caso de normas segecex, bastaria 224 para qtd5 e 891 para qtd1
        print(f"==={cnt}/{len(list_experiment)}===>  {time.strftime('%Y-%b-%d %H:%M:%S')} experiment: {experiment}")
        result_run = util_search.experiment_run(parm_df=df_search_data, 
                                                parm_experiment=experiment,
                                                parm_ndcg_position=12,
                                                parm_limit_query=limit_query,
                                                parm_print=True)
        list_result_experiment.append(result_run)
        # print(f"qtd_encontrado sim:{result_run['qtd_encontrado']}, não:{result_run['qtd_nao_encontrado']} ({result_run['percent_nao_encontrado']}%)")
        experiment['DONE'] = True
        util_search.add_experiment_result(parm_list_result=list_result_experiment, parm_dataset='juris_tcu_index')



===0/8===>  2023-Jun-12 11:37:50 experiment: {'CRITERIA': 'total', 'TOPK_RETRIEVER': 400, 'TOPK_RANKER': 50, 'DONE': False, 'PIPE': {'PIPE_NAME': 'pipe_bm25_ranker_monot5_3b', 'PIPE_OBJECT': <haystack.pipelines.base.Pipeline object at 0x7f3ff38305d0>, 'RETRIEVER_TYPE': 'bm25', 'RETRIEVER_MODEL_NAME': '', 'RANKER_MODEL_NAME': 'unicamp-dl/mt5-3B-mmarco-en-pt'}}


 99%|█████████▉| 99/100 [46:28<00:28, 28.17s/it]


RANK1_MEAN: 3.57
NDCG_MEAN: 39.843
TIME_SPENT_MEAN: 27.889
===1/8===>  2023-Jun-12 12:24:19 experiment: {'CRITERIA': 'total', 'TOPK_RETRIEVER': 300, 'TOPK_RANKER': 50, 'DONE': False, 'PIPE': {'PIPE_NAME': 'pipe_bm25_ranker_monot5_3b', 'PIPE_OBJECT': <haystack.pipelines.base.Pipeline object at 0x7f3ff38305d0>, 'RETRIEVER_TYPE': 'bm25', 'RETRIEVER_MODEL_NAME': '', 'RANKER_MODEL_NAME': 'unicamp-dl/mt5-3B-mmarco-en-pt'}}


 99%|█████████▉| 99/100 [35:27<00:21, 21.49s/it]


RANK1_MEAN: 4.1
NDCG_MEAN: 39.937
TIME_SPENT_MEAN: 21.274
===2/8===>  2023-Jun-12 12:59:47 experiment: {'CRITERIA': 'total', 'TOPK_RETRIEVER': 200, 'TOPK_RANKER': 50, 'DONE': False, 'PIPE': {'PIPE_NAME': 'pipe_bm25_ranker_monot5_3b', 'PIPE_OBJECT': <haystack.pipelines.base.Pipeline object at 0x7f3ff38305d0>, 'RETRIEVER_TYPE': 'bm25', 'RETRIEVER_MODEL_NAME': '', 'RANKER_MODEL_NAME': 'unicamp-dl/mt5-3B-mmarco-en-pt'}}


...

RANK1_MEAN: 3.16
NDCG_MEAN: 39.915
TIME_SPENT_MEAN: 21.299
===6/8===>  2023-Jun-12 14:58:12 experiment: {'CRITERIA': 'total_gte_5', 'TOPK_RETRIEVER': 200, 'TOPK_RANKER': 50, 'DONE': False, 'PIPE': {'PIPE_NAME': 'pipe_bm25_ranker_monot5_3b', 'PIPE_OBJECT': <haystack.pipelines.base.Pipeline object at 0x7f3ff38305d0>, 'RETRIEVER_TYPE': 'bm25', 'RETRIEVER_MODEL_NAME': '', 'RANKER_MODEL_NAME': 'unicamp-dl/mt5-3B-mmarco-en-pt'}}


 99%|█████████▉| 99/100 [23:48<00:14, 14.43s/it]


RANK1_MEAN: 2.81
NDCG_MEAN: 40.249
TIME_SPENT_MEAN: 14.283
===7/8===>  2023-Jun-12 15:22:00 experiment: {'CRITERIA': 'total_gte_5', 'TOPK_RETRIEVER': 100, 'TOPK_RANKER': 50, 'DONE': False, 'PIPE': {'PIPE_NAME': 'pipe_bm25_ranker_monot5_3b', 'PIPE_OBJECT': <haystack.pipelines.base.Pipeline object at 0x7f3ff38305d0>, 'RETRIEVER_TYPE': 'bm25', 'RETRIEVER_MODEL_NAME': '', 'RANKER_MODEL_NAME': 'unicamp-dl/mt5-3B-mmarco-en-pt'}}


 99%|█████████▉| 99/100 [11:59<00:07,  7.27s/it]


RANK1_MEAN: 2.82
NDCG_MEAN: 39.205
TIME_SPENT_MEAN: 7.197
CPU times: user 2h 20min 58s, sys: 1h 33min 51s, total: 3h 54min 50s
Wall time: 3h 56min 9s


## Rodar mono5-3b:sts

### topk_ranker 100

In [ ]:
pipes

[{'PIPE_NAME': 'pipe_sts_ranker_monot5_3b',
  'PIPE_OBJECT': <haystack.pipelines.base.Pipeline at 0x7f45196091d0>,
  'RETRIEVER_TYPE': 'sts',
  'RETRIEVER_MODEL_NAME': 'rufimelo/Legal-BERTimbau-sts-large-ma-v3',
  'RANKER_MODEL_NAME': 'unicamp-dl/mt5-3B-mmarco-en-pt'}]

In [ ]:
grid_experiment = {'CRITERIA' : ['total', 'total_gte_5'],
             'TOPK_RETRIEVER' : [300, 200, 100],
             'TOPK_RANKER' : [100],
             'PIPE': pipes,
             'DONE': [False]
}

In [ ]:
list_experiment = [dict(zip(grid_experiment.keys(), values)) for values in product(*grid_experiment.values())]


In [ ]:
print(len(list_experiment))
gc.collect()

6


110

In [ ]:
import importlib
importlib.reload(util_search)

dict_idcg_relevance_fixed {1: 1.0, 2: 1.6309297535714575, 3: 2.1309297535714578, 4: 2.5616063116448506, 5: 2.9484591188793923, 6: 3.3046663059874146, 7: 3.637999639320748, 8: 3.953464516106477, 9: 4.254494511770458, 10: 4.543559338088346, 11: 4.8225022837394755, 12: 5.092740438166795, 13: 5.355389973203989, 14: 5.611347998013804}


<module 'util.util_search' from '/home/borela/fontes/ind-ir/code/util/util_search.py'>

In [ ]:
%%time
# colocar pequeno só para testes, depois voltar para 9999999 (número maior do que o número de termos a pesquisar)
limit_query = 100 # 9999999
for cnt, experiment in enumerate(list_experiment):
    if not experiment['DONE']:
        list_result_experiment = [] # por experiment
        # no caso de normas segecex, bastaria 224 para qtd5 e 891 para qtd1
        print(f"==={cnt}/{len(list_experiment)}===>  {time.strftime('%Y-%b-%d %H:%M:%S')} experiment: {experiment}")
        result_run = util_search.experiment_run(parm_df=df_search_data, 
                                                parm_experiment=experiment,
                                                parm_ndcg_position=12,
                                                parm_limit_query=limit_query,
                                                parm_print=True)
        list_result_experiment.append(result_run)
        # print(f"qtd_encontrado sim:{result_run['qtd_encontrado']}, não:{result_run['qtd_nao_encontrado']} ({result_run['percent_nao_encontrado']}%)")
        experiment['DONE'] = True
        util_search.add_experiment_result(parm_list_result=list_result_experiment, parm_dataset='juris_tcu_index')



===0/6===>  2023-Jun-12 07:11:01 experiment: {'CRITERIA': 'total', 'TOPK_RETRIEVER': 300, 'TOPK_RANKER': 100, 'PIPE': {'PIPE_NAME': 'pipe_sts_ranker_monot5_3b', 'PIPE_OBJECT': <haystack.pipelines.base.Pipeline object at 0x7f45196091d0>, 'RETRIEVER_TYPE': 'sts', 'RETRIEVER_MODEL_NAME': 'rufimelo/Legal-BERTimbau-sts-large-ma-v3', 'RANKER_MODEL_NAME': 'unicamp-dl/mt5-3B-mmarco-en-pt'}, 'DONE': False}


 99%|█████████▉| 99/100 [33:37<00:20, 20.38s/it]


RANK1_MEAN: 3.81
NDCG_MEAN: 36.022
TIME_SPENT_MEAN: 20.177
===1/6===>  2023-Jun-12 07:44:39 experiment: {'CRITERIA': 'total', 'TOPK_RETRIEVER': 200, 'TOPK_RANKER': 100, 'PIPE': {'PIPE_NAME': 'pipe_sts_ranker_monot5_3b', 'PIPE_OBJECT': <haystack.pipelines.base.Pipeline object at 0x7f45196091d0>, 'RETRIEVER_TYPE': 'sts', 'RETRIEVER_MODEL_NAME': 'rufimelo/Legal-BERTimbau-sts-large-ma-v3', 'RANKER_MODEL_NAME': 'unicamp-dl/mt5-3B-mmarco-en-pt'}, 'DONE': False}


 99%|█████████▉| 99/100 [23:00<00:13, 13.94s/it]


RANK1_MEAN: 3.859
NDCG_MEAN: 35.575
TIME_SPENT_MEAN: 13.801
===2/6===>  2023-Jun-12 08:07:39 experiment: {'CRITERIA': 'total', 'TOPK_RETRIEVER': 100, 'TOPK_RANKER': 100, 'PIPE': {'PIPE_NAME': 'pipe_sts_ranker_monot5_3b', 'PIPE_OBJECT': <haystack.pipelines.base.Pipeline object at 0x7f45196091d0>, 'RETRIEVER_TYPE': 'sts', 'RETRIEVER_MODEL_NAME': 'rufimelo/Legal-BERTimbau-sts-large-ma-v3', 'RANKER_MODEL_NAME': 'unicamp-dl/mt5-3B-mmarco-en-pt'}, 'DONE': False}


...

RANK1_MEAN: 3.54
NDCG_MEAN: 36.67
TIME_SPENT_MEAN: 20.773
===4/6===>  2023-Jun-12 08:53:57 experiment: {'CRITERIA': 'total_gte_5', 'TOPK_RETRIEVER': 200, 'TOPK_RANKER': 100, 'PIPE': {'PIPE_NAME': 'pipe_sts_ranker_monot5_3b', 'PIPE_OBJECT': <haystack.pipelines.base.Pipeline object at 0x7f45196091d0>, 'RETRIEVER_TYPE': 'sts', 'RETRIEVER_MODEL_NAME': 'rufimelo/Legal-BERTimbau-sts-large-ma-v3', 'RANKER_MODEL_NAME': 'unicamp-dl/mt5-3B-mmarco-en-pt'}, 'DONE': False}


 99%|█████████▉| 99/100 [23:14<00:14, 14.09s/it]


RANK1_MEAN: 3.33
NDCG_MEAN: 36.033
TIME_SPENT_MEAN: 13.947
===5/6===>  2023-Jun-12 09:17:11 experiment: {'CRITERIA': 'total_gte_5', 'TOPK_RETRIEVER': 100, 'TOPK_RANKER': 100, 'PIPE': {'PIPE_NAME': 'pipe_sts_ranker_monot5_3b', 'PIPE_OBJECT': <haystack.pipelines.base.Pipeline object at 0x7f45196091d0>, 'RETRIEVER_TYPE': 'sts', 'RETRIEVER_MODEL_NAME': 'rufimelo/Legal-BERTimbau-sts-large-ma-v3', 'RANKER_MODEL_NAME': 'unicamp-dl/mt5-3B-mmarco-en-pt'}, 'DONE': False}


 99%|█████████▉| 99/100 [11:45<00:07,  7.12s/it]

RANK1_MEAN: 2.646
NDCG_MEAN: 35.36
TIME_SPENT_MEAN: 7.05
CPU times: user 1h 19min 45s, sys: 57min 19s, total: 2h 17min 5s
Wall time: 2h 17min 55s


### topk_ranker 50

In [ ]:
pipes

[{'PIPE_NAME': 'pipe_sts_ranker_monot5_3b',
  'PIPE_OBJECT': <haystack.pipelines.base.Pipeline at 0x7f9037d80290>,
  'RETRIEVER_TYPE': 'sts',
  'RETRIEVER_MODEL_NAME': 'rufimelo/Legal-BERTimbau-sts-large-ma-v3',
  'RANKER_MODEL_NAME': 'unicamp-dl/mt5-3B-mmarco-en-pt'}]

In [ ]:
grid_experiment = {
             'CRITERIA' : ['total', 'total_gte_5'],
             'TOPK_RETRIEVER' : [400, 300, 200, 100],
             'TOPK_RANKER' : [50],
             'DONE': [False],
             'PIPE': pipes,
}

In [ ]:
list_experiment = [dict(zip(grid_experiment.keys(), values)) for values in product(*grid_experiment.values())]


In [ ]:
print(len(list_experiment))
gc.collect()

8


101

In [ ]:
import importlib
importlib.reload(util_search)

dict_idcg_relevance_fixed {1: 1.0, 2: 1.6309297535714575, 3: 2.1309297535714578, 4: 2.5616063116448506, 5: 2.9484591188793923, 6: 3.3046663059874146, 7: 3.637999639320748, 8: 3.953464516106477, 9: 4.254494511770458, 10: 4.543559338088346, 11: 4.8225022837394755, 12: 5.092740438166795, 13: 5.355389973203989, 14: 5.611347998013804}


<module 'util.util_search' from '/home/borela/fontes/ind-ir/code/util/util_search.py'>

In [ ]:
%%time
# colocar pequeno só para testes, depois voltar para 9999999 (número maior do que o número de termos a pesquisar)
limit_query = 100 # 9999999
for cnt, experiment in enumerate(list_experiment):
    if not experiment['DONE']:
        list_result_experiment = [] # por experiment
        # no caso de normas segecex, bastaria 224 para qtd5 e 891 para qtd1
        print(f"==={cnt}/{len(list_experiment)}===>  {time.strftime('%Y-%b-%d %H:%M:%S')} experiment: {experiment}")
        result_run = util_search.experiment_run(parm_df=df_search_data, 
                                                parm_experiment=experiment,
                                                parm_ndcg_position=12,
                                                parm_limit_query=limit_query,
                                                parm_print=True)
        list_result_experiment.append(result_run)
        # print(f"qtd_encontrado sim:{result_run['qtd_encontrado']}, não:{result_run['qtd_nao_encontrado']} ({result_run['percent_nao_encontrado']}%)")
        experiment['DONE'] = True
        util_search.add_experiment_result(parm_list_result=list_result_experiment, parm_dataset='juris_tcu_index')


===0/8===>  2023-Jun-12 17:05:59 experiment: {'CRITERIA': 'total', 'TOPK_RETRIEVER': 400, 'TOPK_RANKER': 50, 'DONE': False, 'PIPE': {'PIPE_NAME': 'pipe_sts_ranker_monot5_3b', 'PIPE_OBJECT': <haystack.pipelines.base.Pipeline object at 0x7f9037d80290>, 'RETRIEVER_TYPE': 'sts', 'RETRIEVER_MODEL_NAME': 'rufimelo/Legal-BERTimbau-sts-large-ma-v3', 'RANKER_MODEL_NAME': 'unicamp-dl/mt5-3B-mmarco-en-pt'}}


 99%|█████████▉| 99/100 [44:46<00:27, 27.13s/it]


RANK1_MEAN: 3.87
NDCG_MEAN: 36.152
TIME_SPENT_MEAN: 26.863
===1/8===>  2023-Jun-12 17:50:46 experiment: {'CRITERIA': 'total', 'TOPK_RETRIEVER': 300, 'TOPK_RANKER': 50, 'DONE': False, 'PIPE': {'PIPE_NAME': 'pipe_sts_ranker_monot5_3b', 'PIPE_OBJECT': <haystack.pipelines.base.Pipeline object at 0x7f9037d80290>, 'RETRIEVER_TYPE': 'sts', 'RETRIEVER_MODEL_NAME': 'rufimelo/Legal-BERTimbau-sts-large-ma-v3', 'RANKER_MODEL_NAME': 'unicamp-dl/mt5-3B-mmarco-en-pt'}}


 99%|█████████▉| 99/100 [34:07<00:20, 20.68s/it]


RANK1_MEAN: 3.81
NDCG_MEAN: 36.022
TIME_SPENT_MEAN: 20.474
===2/8===>  2023-Jun-12 18:24:53 experiment: {'CRITERIA': 'total', 'TOPK_RETRIEVER': 200, 'TOPK_RANKER': 50, 'DONE': False, 'PIPE': {'PIPE_NAME': 'pipe_sts_ranker_monot5_3b', 'PIPE_OBJECT': <haystack.pipelines.base.Pipeline object at 0x7f9037d80290>, 'RETRIEVER_TYPE': 'sts', 'RETRIEVER_MODEL_NAME': 'rufimelo/Legal-BERTimbau-sts-large-ma-v3', 'RANKER_MODEL_NAME': 'unicamp-dl/mt5-3B-mmarco-en-pt'}}


...

RANK1_MEAN: 3.54
NDCG_MEAN: 36.67
TIME_SPENT_MEAN: 20.772
===6/8===>  2023-Jun-12 20:19:40 experiment: {'CRITERIA': 'total_gte_5', 'TOPK_RETRIEVER': 200, 'TOPK_RANKER': 50, 'DONE': False, 'PIPE': {'PIPE_NAME': 'pipe_sts_ranker_monot5_3b', 'PIPE_OBJECT': <haystack.pipelines.base.Pipeline object at 0x7f9037d80290>, 'RETRIEVER_TYPE': 'sts', 'RETRIEVER_MODEL_NAME': 'rufimelo/Legal-BERTimbau-sts-large-ma-v3', 'RANKER_MODEL_NAME': 'unicamp-dl/mt5-3B-mmarco-en-pt'}}


 99%|█████████▉| 99/100 [23:12<00:14, 14.06s/it]


RANK1_MEAN: 3.33
NDCG_MEAN: 36.033
TIME_SPENT_MEAN: 13.921
===7/8===>  2023-Jun-12 20:42:53 experiment: {'CRITERIA': 'total_gte_5', 'TOPK_RETRIEVER': 100, 'TOPK_RANKER': 50, 'DONE': False, 'PIPE': {'PIPE_NAME': 'pipe_sts_ranker_monot5_3b', 'PIPE_OBJECT': <haystack.pipelines.base.Pipeline object at 0x7f9037d80290>, 'RETRIEVER_TYPE': 'sts', 'RETRIEVER_MODEL_NAME': 'rufimelo/Legal-BERTimbau-sts-large-ma-v3', 'RANKER_MODEL_NAME': 'unicamp-dl/mt5-3B-mmarco-en-pt'}}


 99%|█████████▉| 99/100 [11:42<00:07,  7.10s/it]


RANK1_MEAN: 2.646
NDCG_MEAN: 35.36
TIME_SPENT_MEAN: 7.024
CPU times: user 2h 8min 33s, sys: 1h 38min 43s, total: 3h 47min 16s
Wall time: 3h 48min 36s


## Rodar minilm: bm25 e sts

In [ ]:
pipes

[{'PIPE_NAME': 'pipe_bm25_ranker_minilm',
  'PIPE_OBJECT': <haystack.pipelines.base.Pipeline at 0x7f25f350d3d0>,
  'RETRIEVER_TYPE': 'bm25',
  'RETRIEVER_MODEL_NAME': '',
  'RANKER_MODEL_NAME': 'unicamp-dl/mMiniLM-L6-v2-pt-v2'},
 {'PIPE_NAME': 'pipe_sts_ranker_minilm',
  'PIPE_OBJECT': <haystack.pipelines.base.Pipeline at 0x7f25f1badf10>,
  'RETRIEVER_TYPE': 'sts',
  'RETRIEVER_MODEL_NAME': 'rufimelo/Legal-BERTimbau-sts-large-ma-v3',
  'RANKER_MODEL_NAME': 'unicamp-dl/mMiniLM-L6-v2-pt-v2'}]

In [ ]:
grid_experiment = {'CRITERIA' : ['total', 'total_gte_5'],
             'TOPK_RETRIEVER' : [300, 200, 100],
             'TOPK_RANKER' : [100],
             'PIPE': pipes,
             'DONE': [False]
}

In [ ]:
list_experiment = [dict(zip(grid_experiment.keys(), values)) for values in product(*grid_experiment.values())]


In [ ]:
print(len(list_experiment))
gc.collect()

12


63

In [ ]:
import importlib
importlib.reload(util_search)

<module 'util.util_search' from '/home/borela/fontes/ind-ir/code/util/util_search.py'>

In [ ]:
%%time
# colocar pequeno só para testes, depois voltar para 9999999 (número maior do que o número de termos a pesquisar)
limit_query = 100 # 9999999
for cnt, experiment in enumerate(list_experiment):
    if not experiment['DONE']:
        list_result_experiment = [] # por experiment
        # no caso de normas segecex, bastaria 224 para qtd5 e 891 para qtd1
        print(f"==={cnt}/{len(list_experiment)}===>  {time.strftime('%Y-%b-%d %H:%M:%S')} experiment: {experiment}")
        result_run = util_search.experiment_run(parm_df=df_search_data, 
                                                parm_experiment=experiment,
                                                parm_ndcg_position=12,
                                                parm_limit_query=limit_query,
                                                parm_print=True)
        list_result_experiment.append(result_run)
        # print(f"qtd_encontrado sim:{result_run['qtd_encontrado']}, não:{result_run['qtd_nao_encontrado']} ({result_run['percent_nao_encontrado']}%)")
        experiment['DONE'] = True
        util_search.add_experiment_result(parm_list_result=list_result_experiment, parm_dataset='juris_tcu_index')



===0/12===>  2023-Jun-12 10:02:16 experiment: {'CRITERIA': 'total', 'TOPK_RETRIEVER': 300, 'TOPK_RANKER': 100, 'PIPE': {'PIPE_NAME': 'pipe_bm25_ranker_minilm', 'PIPE_OBJECT': <haystack.pipelines.base.Pipeline object at 0x7f25f350d3d0>, 'RETRIEVER_TYPE': 'bm25', 'RETRIEVER_MODEL_NAME': '', 'RANKER_MODEL_NAME': 'unicamp-dl/mMiniLM-L6-v2-pt-v2'}, 'DONE': False}


 99%|█████████▉| 99/100 [01:20<00:00,  1.23it/s]


RANK1_MEAN: 8.626
NDCG_MEAN: 24.26
TIME_SPENT_MEAN: 0.807
===1/12===>  2023-Jun-12 10:03:37 experiment: {'CRITERIA': 'total', 'TOPK_RETRIEVER': 300, 'TOPK_RANKER': 100, 'PIPE': {'PIPE_NAME': 'pipe_sts_ranker_minilm', 'PIPE_OBJECT': <haystack.pipelines.base.Pipeline object at 0x7f25f1badf10>, 'RETRIEVER_TYPE': 'sts', 'RETRIEVER_MODEL_NAME': 'rufimelo/Legal-BERTimbau-sts-large-ma-v3', 'RANKER_MODEL_NAME': 'unicamp-dl/mMiniLM-L6-v2-pt-v2'}, 'DONE': False}


 99%|█████████▉| 99/100 [01:20<00:00,  1.23it/s]


RANK1_MEAN: 8.505
NDCG_MEAN: 22.769
TIME_SPENT_MEAN: 0.807
===2/12===>  2023-Jun-12 10:04:57 experiment: {'CRITERIA': 'total', 'TOPK_RETRIEVER': 200, 'TOPK_RANKER': 100, 'PIPE': {'PIPE_NAME': 'pipe_bm25_ranker_minilm', 'PIPE_OBJECT': <haystack.pipelines.base.Pipeline object at 0x7f25f350d3d0>, 'RETRIEVER_TYPE': 'bm25', 'RETRIEVER_MODEL_NAME': '', 'RANKER_MODEL_NAME': 'unicamp-dl/mMiniLM-L6-v2-pt-v2'}, 'DONE': False}


...

RANK1_MEAN: 8.475
NDCG_MEAN: 23.86
TIME_SPENT_MEAN: 0.552
===10/12===>  2023-Jun-12 10:12:19 experiment: {'CRITERIA': 'total_gte_5', 'TOPK_RETRIEVER': 100, 'TOPK_RANKER': 100, 'PIPE': {'PIPE_NAME': 'pipe_bm25_ranker_minilm', 'PIPE_OBJECT': <haystack.pipelines.base.Pipeline object at 0x7f25f350d3d0>, 'RETRIEVER_TYPE': 'bm25', 'RETRIEVER_MODEL_NAME': '', 'RANKER_MODEL_NAME': 'unicamp-dl/mMiniLM-L6-v2-pt-v2'}, 'DONE': False}


 99%|█████████▉| 99/100 [00:28<00:00,  3.52it/s]


RANK1_MEAN: 7.04
NDCG_MEAN: 25.979
TIME_SPENT_MEAN: 0.281
===11/12===>  2023-Jun-12 10:12:47 experiment: {'CRITERIA': 'total_gte_5', 'TOPK_RETRIEVER': 100, 'TOPK_RANKER': 100, 'PIPE': {'PIPE_NAME': 'pipe_sts_ranker_minilm', 'PIPE_OBJECT': <haystack.pipelines.base.Pipeline object at 0x7f25f1badf10>, 'RETRIEVER_TYPE': 'sts', 'RETRIEVER_MODEL_NAME': 'rufimelo/Legal-BERTimbau-sts-large-ma-v3', 'RANKER_MODEL_NAME': 'unicamp-dl/mMiniLM-L6-v2-pt-v2'}, 'DONE': False}


 99%|█████████▉| 99/100 [00:29<00:00,  3.41it/s]


RANK1_MEAN: 5.865
NDCG_MEAN: 23.582
TIME_SPENT_MEAN: 0.29
CPU times: user 12min 22s, sys: 3.99 s, total: 12min 26s
Wall time: 11min


In [ ]:
pipes

[{'PIPE_NAME': 'pipe_bm25_ranker_minilm',
  'PIPE_OBJECT': <haystack.pipelines.base.Pipeline at 0x7f25f350d3d0>,
  'RETRIEVER_TYPE': 'bm25',
  'RETRIEVER_MODEL_NAME': '',
  'RANKER_MODEL_NAME': 'unicamp-dl/mMiniLM-L6-v2-pt-v2'},
 {'PIPE_NAME': 'pipe_sts_ranker_minilm',
  'PIPE_OBJECT': <haystack.pipelines.base.Pipeline at 0x7f25f1badf10>,
  'RETRIEVER_TYPE': 'sts',
  'RETRIEVER_MODEL_NAME': 'rufimelo/Legal-BERTimbau-sts-large-ma-v3',
  'RANKER_MODEL_NAME': 'unicamp-dl/mMiniLM-L6-v2-pt-v2'}]

In [ ]:
grid_experiment = {'CRITERIA' : ['total', 'total_gte_5'],
             'TOPK_RETRIEVER' : [400, 300, 200, 100],
             'TOPK_RANKER' : [200, 50],
             'PIPE': pipes,
             'DONE': [False]
}

In [ ]:
list_experiment = [dict(zip(grid_experiment.keys(), values)) for values in product(*grid_experiment.values())]


In [ ]:
print(len(list_experiment))
gc.collect()

32


84

In [ ]:
import importlib
importlib.reload(util_search)

<module 'util.util_search' from '/home/borela/fontes/ind-ir/code/util/util_search.py'>

In [ ]:
%%time
# colocar pequeno só para testes, depois voltar para 9999999 (número maior do que o número de termos a pesquisar)
limit_query = 100 # 9999999
for cnt, experiment in enumerate(list_experiment):
    if not experiment['DONE']:
        list_result_experiment = [] # por experiment
        # no caso de normas segecex, bastaria 224 para qtd5 e 891 para qtd1
        print(f"==={cnt}/{len(list_experiment)}===>  {time.strftime('%Y-%b-%d %H:%M:%S')} experiment: {experiment}")
        result_run = util_search.experiment_run(parm_df=df_search_data, 
                                                parm_experiment=experiment,
                                                parm_ndcg_position=12,
                                                parm_limit_query=limit_query,
                                                parm_print=True)
        list_result_experiment.append(result_run)
        # print(f"qtd_encontrado sim:{result_run['qtd_encontrado']}, não:{result_run['qtd_nao_encontrado']} ({result_run['percent_nao_encontrado']}%)")
        experiment['DONE'] = True
        util_search.add_experiment_result(parm_list_result=list_result_experiment, parm_dataset='juris_tcu_index')



===0/32===>  2023-Jun-12 10:13:17 experiment: {'CRITERIA': 'total', 'TOPK_RETRIEVER': 400, 'TOPK_RANKER': 200, 'PIPE': {'PIPE_NAME': 'pipe_bm25_ranker_minilm', 'PIPE_OBJECT': <haystack.pipelines.base.Pipeline object at 0x7f25f350d3d0>, 'RETRIEVER_TYPE': 'bm25', 'RETRIEVER_MODEL_NAME': '', 'RANKER_MODEL_NAME': 'unicamp-dl/mMiniLM-L6-v2-pt-v2'}, 'DONE': False}


 99%|█████████▉| 99/100 [01:48<00:01,  1.10s/it]


RANK1_MEAN: 11.13
NDCG_MEAN: 24.237
TIME_SPENT_MEAN: 1.085
===1/32===>  2023-Jun-12 10:15:06 experiment: {'CRITERIA': 'total', 'TOPK_RETRIEVER': 400, 'TOPK_RANKER': 200, 'PIPE': {'PIPE_NAME': 'pipe_sts_ranker_minilm', 'PIPE_OBJECT': <haystack.pipelines.base.Pipeline object at 0x7f25f1badf10>, 'RETRIEVER_TYPE': 'sts', 'RETRIEVER_MODEL_NAME': 'rufimelo/Legal-BERTimbau-sts-large-ma-v3', 'RANKER_MODEL_NAME': 'unicamp-dl/mMiniLM-L6-v2-pt-v2'}, 'DONE': False}


 99%|█████████▉| 99/100 [01:47<00:01,  1.09s/it]


RANK1_MEAN: 11.848
NDCG_MEAN: 23.229
TIME_SPENT_MEAN: 1.075
===2/32===>  2023-Jun-12 10:16:53 experiment: {'CRITERIA': 'total', 'TOPK_RETRIEVER': 400, 'TOPK_RANKER': 50, 'PIPE': {'PIPE_NAME': 'pipe_bm25_ranker_minilm', 'PIPE_OBJECT': <haystack.pipelines.base.Pipeline object at 0x7f25f350d3d0>, 'RETRIEVER_TYPE': 'bm25', 'RETRIEVER_MODEL_NAME': '', 'RANKER_MODEL_NAME': 'unicamp-dl/mMiniLM-L6-v2-pt-v2'}, 'DONE': False}


...

RANK1_MEAN: 5.865
NDCG_MEAN: 23.582
TIME_SPENT_MEAN: 0.291
===30/32===>  2023-Jun-12 10:48:47 experiment: {'CRITERIA': 'total_gte_5', 'TOPK_RETRIEVER': 100, 'TOPK_RANKER': 50, 'PIPE': {'PIPE_NAME': 'pipe_bm25_ranker_minilm', 'PIPE_OBJECT': <haystack.pipelines.base.Pipeline object at 0x7f25f350d3d0>, 'RETRIEVER_TYPE': 'bm25', 'RETRIEVER_MODEL_NAME': '', 'RANKER_MODEL_NAME': 'unicamp-dl/mMiniLM-L6-v2-pt-v2'}, 'DONE': False}


 99%|█████████▉| 99/100 [00:27<00:00,  3.54it/s]


RANK1_MEAN: 5.867
NDCG_MEAN: 25.979
TIME_SPENT_MEAN: 0.279
===31/32===>  2023-Jun-12 10:49:15 experiment: {'CRITERIA': 'total_gte_5', 'TOPK_RETRIEVER': 100, 'TOPK_RANKER': 50, 'PIPE': {'PIPE_NAME': 'pipe_sts_ranker_minilm', 'PIPE_OBJECT': <haystack.pipelines.base.Pipeline object at 0x7f25f1badf10>, 'RETRIEVER_TYPE': 'sts', 'RETRIEVER_MODEL_NAME': 'rufimelo/Legal-BERTimbau-sts-large-ma-v3', 'RANKER_MODEL_NAME': 'unicamp-dl/mMiniLM-L6-v2-pt-v2'}, 'DONE': False}


 99%|█████████▉| 99/100 [00:28<00:00,  3.43it/s]


RANK1_MEAN: 5.305
NDCG_MEAN: 23.582
TIME_SPENT_MEAN: 0.289
CPU times: user 40min 58s, sys: 10.7 s, total: 41min 9s
Wall time: 36min 27s
